In [1]:
# Copyright (c) 2024 Graphcore Ltd. All rights reserved.

## Brief Tour of MESS

MESS is a modular toolkit for exploring the exciting interface between machine
learning, electronic structure, and algorithms.

To begin our tour we build a single water molecule.
Each atom is represented an atomic number $Z_i$ and a position in Cartesian
coordinates $(x_i, y_i, z_i)$.  In MESS we collect atoms into a `Structure` and we
provide a few examples built by the `molecule` function.  
MESS is designed for interactive exploration so in a notebook environment a `Structure`
object will display a 3D visualisation

In [2]:
from mess import molecule

mol = molecule("water")
mol

/home/ubuntu/miniforge3/envs/jax/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Structure(atomic_number=i64[3](numpy), position=f64[3,3](numpy))

MESS represents electrons using the Linear Combination of Atomic Orbitals ([LCAO](https://en.wikipedia.org/wiki/Linear_combination_of_atomic_orbitals)) method.
We rely on the [Basis Set Exchange](https://www.basissetexchange.org/) project to
provide access to the full range of previously calculated Gaussian Type Orbital parameters.

In [3]:
from mess import basisset

basis_name = "6-31g"
basis = basisset(mol, basis_name)
basis

  primitive    orbital    coefficient        norm  center                              lmn            alpha
-----------  ---------  -------------  ----------  ----------------------------------  -------  -----------
          0          0     0.00183107  454.227     [0.        0.        0.2201531]     [0 0 0]  5484.67
          1          0     0.0139502   109.735     [0.        0.        0.2201531]     [0 0 0]   825.235
          2          0     0.0684451    36.1918    [0.        0.        0.2201531]     [0 0 0]   188.047
          3          0     0.232714     13.9926    [0.        0.        0.2201531]     [0 0 0]    52.9645
          4          0     0.470193      5.93989   [0.        0.        0.2201531]     [0 0 0]    16.8976
          5          0     0.358521      2.66355   [0.        0.        0.2201531]     [0 0 0]     5.79964
          6          1    -0.110778      5.57815   [0.        0.        0.2201531]     [0 0 0]    15.5396
          7          1    -0.148026      1.86

We now have all the pieces to run our first electronic structure simulation.
This is done in two steps:
* build a Hamiltonian by selecting a treatment for the quantum-mechanical exchange and correlation.
* find the molecular orbital coefficients $C$ that minimise the energy of this Hamiltonian

In the following we use the [Local Density Approximation](https://en.wikipedia.org/wiki/Local-density_approximation) (LDA) of Density Functional Theory (DFT) to model the quantum-mechanical electron interactions.

By default the minimisation routine will be compiled by JAX and executed on any available
hardware accelerator (e.g. GPU/TPU).

In [4]:
from mess import minimise, Hamiltonian

H = Hamiltonian(basis, xc_method="lda")
E, C, sol = minimise(H)
float(E)

-76.0142778342404

We can visualise the electron density $\rho(\mathbf{r})$ from the solution we found.
The electron cloud is approximately mickey mouse head shaped to use the technical term.

In [5]:
import py3Dmol
from mess.mesh import density, uniform_mesh
from mess.plot import plot_volume, plot_molecule

view = py3Dmol.view()
plot_molecule(view, mol)
mesh = uniform_mesh()
rho = density(basis, mesh, basis.density_matrix(C))
plot_volume(view, rho, mesh.axes)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Performance is one of several goals of the MESS project so lets measure how long the
energy minimisation takes

In [6]:
%%timeit
E, C, _ = minimise(H)
E, C = E.block_until_ready(), C.block_until_ready()

182 ms ± 615 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Before we get carried away profiling, we first make sure that the MESS simulation agrees with a standard
and well used DFT software package - [PySCF](https://pyscf.org/)

In [7]:
from mess.interop import to_pyscf
from pyscf import dft, scf


scf_mol = to_pyscf(mol, basis_name)
s = dft.RKS(scf_mol, xc="lda,vwn_rpa")
s.kernel()

/home/ubuntu/miniforge3/envs/jax/lib/python3.10/site-packages/pyscf/gto/mole.py:1280: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -76.0142755653013


-76.01427556530132

The calculated energies match!...lets open a can of worms and measure the performance of
 the PySCF energy minimisation

In [8]:
%%timeit
s.kernel()

converged SCF energy = -76.0142755653013
converged SCF energy = -76.0142755653013
converged SCF energy = -76.0142755653013
converged SCF energy = -76.0142755653012
converged SCF energy = -76.0142755653012
converged SCF energy = -76.0142755653012
converged SCF energy = -76.0142755653013
converged SCF energy = -76.0142755653012
1.53 s ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We've measured a nearly 8X speedup...There are several gotchas of course that will
explored in due time...stay tuned for more!

## Hartree-Fock

Hartree-Fock is a closely related method to the DFT solution found above that in MESS
is selected by passing `xc_method=hfx`

In [9]:
H = Hamiltonian(basis, xc_method="hfx")
E, C, sol = minimise(H)
float(E)

-75.9841735383759

The energy is a little higher than the DFT solution found earlier but at a significantly 
reduced computational cost

In [10]:
%%timeit
minimise(H)

29.2 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As another sanity check we make sure the calculated Hartree-Fock energy calculated by MESS agrees with PySCF

In [11]:
s = scf.RHF(scf_mol)
s.kernel()

/home/ubuntu/miniforge3/envs/jax/lib/python3.10/site-packages/pyscf/gto/mole.py:1280: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -75.9841721516934


-75.98417215169337

We hope you enjoyed your tour of MESS and welcome any feedback as
[github issues](https://github.com/graphcore-research/mess/issues) where we can continue the discussion.


:::{note}
For reproducibility we record the default accelerator (if any) used by JAX and the CPU architecture used to execute this notebook.
:::

In [12]:
import jax

jax.devices()[0].device_kind

'NVIDIA A10G'

In [13]:
!lscpu

Architecture:                       x86_64
CPU op-mode(s):                     32-bit, 64-bit
Byte Order:                         Little Endian
Address sizes:                      48 bits physical, 48 bits virtual
CPU(s):                             8
On-line CPU(s) list:                0-7
Thread(s) per core:                 2
Core(s) per socket:                 4
Socket(s):                          1
NUMA node(s):                       1
Vendor ID:                          AuthenticAMD
CPU family:                         23
Model:                              49
Model name:                         AMD EPYC 7R32
Stepping:                           0
CPU MHz:                            3294.507
BogoMIPS:                           5600.00
Hypervisor vendor:                  KVM
Virtualization type:                full
L1d cache:                          128 KiB
L1i cache:                          128 KiB
L2 cache:                           2 MiB
L3 cache:                           16 Mi

/home/ubuntu/miniforge3/envs/jax/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
